### 1. Preprocess the JSON string to fix the data quality issues.

### 2. Transform JSON string to JSON object.

### 3. Write transformed data to Silver schema.



In [0]:
select * from gizmobox.bronze.v_orders;

### 1. Preprocess the JSON string to fix the data quality issues.

In [0]:
SELECT value,
      regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"') AS fixed_value
FROM gizmobox.bronze.v_orders;

In [0]:
CREATE OR REPLACE TEMPORARY VIEW tv_orders_fixed 
AS
SELECT value,
      regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"') AS fixed_value
FROM gizmobox.bronze.v_orders;

### 2. Transform JSON string to JSON object.

In [0]:
SELECT schema_of_json(fixed_value) as json_schema,
       fixed_value
FROM tv_orders_fixed LIMIT 1;

### 3. Write transformed data to Silver schema.

In [0]:
SELECT 
      from_json(fixed_value, 'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') AS json_value,
      fixed_value
FROM tv_orders_fixed;

In [0]:
CREATE OR REPLACE TABLE gizmobox.silver.orders_json
AS
SELECT 
      from_json(fixed_value, 'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') AS json_value
FROM tv_orders_fixed;

In [0]:
SELECT * FROM gizmobox.silver.orders_json;

In [0]:
SELECT
      json_value.customer_id,
      json_value.items.category,
      json_value.items.details,
      json_value.items.item_id,
       json_value.order_date,
       json_value.order_id,
       json_value.order_status,
       json_value.payment_method,
       json_value.total_amount,
       json_value.transaction_timestamp
FROM gizmobox.silver.orders_json;

In [0]:
SELECT
      json_value.customer_id,
       json_value.order_date,
       json_value.order_id,
       json_value.order_status,
       json_value.payment_method,
       json_value.total_amount,
       json_value.transaction_timestamp,
       explode(array_distinct(json_value.items)) AS items
FROM gizmobox.silver.orders_json;

In [0]:
CREATE OR REPLACE TEMPORARY VIEW tv_orders_exploded 
AS
SELECT
      json_value.customer_id,
       json_value.order_date,
       json_value.order_id,
       json_value.order_status,
       json_value.payment_method,
       json_value.total_amount,
       json_value.transaction_timestamp,
       explode(array_distinct(json_value.items)) AS items
FROM gizmobox.silver.orders_json;

In [0]:
SELECT * FROM tv_orders_exploded;

In [0]:
SELECT  
      customer_id,
      order_date,
      order_id,
      order_status,
      payment_method,
      total_amount,
      transaction_timestamp,
      items.item_id,
      items.name,
      items.category,
      items.details.brand,
      items.details.color,
      items.quantity,
      items.price
FROM tv_orders_exploded;

In [0]:
CREATE OR REPLACE TABLE gizmobox.silver.orders
AS
SELECT  
      customer_id,
      order_date,
      order_id,
      order_status,
      payment_method,
      total_amount,
      transaction_timestamp,
      items.item_id,
      items.name,
      items.category,
      items.details.brand,
      items.details.color,
      items.quantity,
      items.price
FROM tv_orders_exploded;

In [0]:
SELECT * FROM gizmobox.silver.orders;